In [1]:
 require 'rest-client'

  def fetch(url, headers = {accept: "*/*"}, user = "", pass="")
  
    response = RestClient::Request.execute({
      method: :get,
      url: url.to_s,
      user: user,
      password: pass,
      headers: headers})

  return response
  
  rescue RestClient::ExceptionWithResponse => e
    $stderr.puts e.inspect
    response = false
    return response  
  rescue RestClient::Exception => e
    $stderr.puts e.inspect
    response = false
    return response  
  rescue Exception => e
    $stderr.puts e.inspect
    response = false
    return response  
    
end 

:fetch

In [2]:
class ListNetworks
  
  @@all_interactions = Array.new
  attr_accessor :network_ID
  attr_accessor :interactors
  attr_accessor :network_go
  attr_accessor :network_kegg
  
  def initialize (params = {})
    @network_ID = params.fetch(:network_ID, "000000000")
    @interactors = params.fetch(:interactors, Array.new)
    @network_go = params.fetch(:network_go, Array.new)
    @network_kegg = params.fetch(:network_kegg, Array.new)
    
    @@all_interactions << self
    
  end
    
  def ListNetworks.is_in_the_list(gene_to_check) 
    if $list_of_genes.include?("#{gene_to_check}") 
      if @network_ID == nil
        ListNetworks.new(:network_ID => $AGI)
        @interactors |= [gene_to_check]
        @network_go |= [InteractionNetwork.find_go(gene_to_check)]
        @network_kegg |= [InteractionNetwork.find_kegg(gene_to_check)]
      elsif @network_ID.include?($AGI) and (not @interactors.include?(gene_to_check))
        @interactors << [gene_to_check]
        @network_go << InteractionNetwork.find_go(gene_to_check)
        @network_kegg << InteractionNetwork.find_kegg(gene_to_check)
      else
        ListNetworks.new(:network_ID => $AGI)
        @interactors << [gene_to_check]
        @network_go << InteractionNetwork.find_go(gene_to_check)
        @network_kegg << InteractionNetwork.find_kegg(gene_to_check)
      end
    end
  end
end

:is_in_the_list

In [3]:
class InteractionNetwork
  
  attr_accessor :gene_ID  
  attr_accessor :interactions
  attr_accessor :go_ID
  attr_accessor :kegg_pathways
  
  
  def initialize (params = {})
    
    @gene_ID = params.fetch(:gene_ID, "000000000")
    @interactions = params.fetch(:interactions, Array.new)
    @go_bio = params.fetch(:go_bio, Array.new)
    @kegg_pathways = params.fetch(:kegg_pathways, Array.new)
    
    $AGI = @gene_ID
    
  end
  
  def InteractionNetwork.is_agi
    return @gene_ID
  end
  

  
  def InteractionNetwork.find_go(gene1)
    res = fetch("http://togows.org/entry/ebi-uniprot/#{gene1}/dr.json");
    if res
      data = JSON.parse(res.body)
      final_go = []
      if data[0] and data[0]["GO"]
        data[0]["GO"].each do |go|
          if go[1].match(/P:/)
            final_go << go[0] + " " + go[1]
            puts final_go
          else 
            next
          end      
        end
      end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end
  
  def InteractionNetwork.find_kegg(gene2)
    ros = fetch("http://togows.org/entry/kegg-genes/ath:#{gene2}/pathways.json");
    if ros
      data = JSON.parse(ros.body)
      final_kegg = []
      if data[0]
        data[0].each do |kegg|
          if kegg[0].match(/ath/)
            final_kegg << kegg[0]+":"+kegg[1]
            puts final_kegg       
          else 
            next
          end
        end
      end
    else
      puts "togows.org failed - see STDERR for details..."
    end
  end
  
  def search_interactions(some_gene, depth) # Function to search the interaction
    if depth <= 2
      depth = depth + 1
      link = fetch("http://bar.utoronto.ca:9090/psicquic/webservices/current/search/query/#{some_gene}"); # Link to search for interctions indicated in http://bar.utoronto.ca/affydb/BAR_instructions.html#AIV
      if link 
        body = link.to_s.split("\n")
        body.each do |interaction|
          line = interaction.split("\t")
          agi1 = line[2].split(":")[1].upcase
          agi2 = line[3].split(":")[1].upcase
          tax1 = line[9].split(":")[1]
          tax2 = line[10].split(":")[1]
          score = line[14].split(":")[1]
          if tax1.to_i != 3702 or tax2.to_i !=3702 # Filtering by species (Arabidopsis thaliana).
            next
          elsif score.to_f < 0.45 # Filtering by an intermediate score (DOI: 10.1093/database/bau131).
            next
          elsif some_gene.to_s == agi1.to_s and some_gene.to_s == agi2.to_s # Discarding interactions beteen the same gene.         
            next
          elsif not (agi1.match(/AT\dG\d\d\d\d\d/)) or not (agi2.match(/AT\dG\d\d\d\d\d/))
            next
          else
            case # Checking the position of the query gene in the "interaction vector"
            when agi1.to_s == some_gene.to_s 
              ggi = [some_gene, agi2]
              ggi_inverse = [agi2, some_gene]
              if not (@interactions.include?(ggi)) and not (@interactions.include?(ggi_inverse)) and agi2 != @gene_ID # Checking that the interaction list does not include this interaction
                @interactions << ggi
                puts @interactions
                @go_bio << InteractionNetwork.find_go(agi2)
                puts @go_bio
                @kegg_pathways << InteractionNetwork.find_kegg(agi2)
                puts @kegg_pathways
                ListNetworks.is_in_the_list(agi2)
                search_interactions(agi2, depth) # Applying recursivity to search for interactions of the genes linked to the input genes
              end
            else
              ggi = [some_gene, agi1]
              ggi_inverse = [agi1, some_gene]
              if not (@interactions.include?(ggi)) and not (@interactions.include?(ggi_inverse)) and agi1 != @gene_ID # Checking that the interaction list does not include this interaction
                @interactions << ggi #  podríamos hacer un flatten! y un uniq
                puts @interactions
                @go_bio << InteractionNetwork.find_go(agi1)
                puts @go_bio
                @kegg_pathways << InteractionNetwork.find_kegg(agi1)
                puts @kegg_pathways
                ListNetworks.is_in_the_list(agi1)
                search_interactions(agi1, depth) # Applying recursivity to search for interactions of the genes linked to the input genes
              end
            end
          end
        end
      else
        puts "bar.utoronto.ca call failed - see STDERR for details..."
    end
  end
end
end

:search_interactions

In [4]:
class New < InteractionNetwork
def New.to_s
    """Gene_ID: #{self.gene_ID}
Interactors: #{self.interactions}
GO_Biological_Process: #{self.go_bio}
KEGG Pathways: #{self.kegg_pathways}"""
  end
end

:to_s

In [ ]:
$list_of_genes = []
input = File.open("Arabidopsis_prueba2.txt", "r").each do |gene_id|
  $list_of_genes << gene_id.strip.upcase
  genes = InteractionNetwork.new(:gene_ID => gene_id.strip.upcase)
  genes.search_interactions(genes.gene_ID,1)
end
input.close

output = File.open("Output.txt", mode: "w")
output << "Assignment 2\nLists of input genes networks that interact with other input genes:\n"
output.close

add = File.open("Output.txt", "a")
add.puts New.to_s
add.close

[["AT1G07010", "AT4G17030"]]
["GO:0019953 P:sexual reproduction"]
["GO:0019953 P:sexual reproduction", "GO:0019953 P:sexual reproduction"]
["GO:0019953 P:sexual reproduction", "GO:0019953 P:sexual reproduction", "GO:0019953 P:sexual reproduction"]
[[["GO:0005576", "C:extracellular region", "IEA:UniProtKB-SubCell"], ["GO:0019953", "P:sexual reproduction", "IEA:InterPro"], ["GO:0005576", "C:extracellular region", "IEA:InterPro"], ["GO:0019953", "P:sexual reproduction", "IEA:InterPro"], ["GO:0016021", "C:integral component of membrane", "IEA:UniProtKB-KW"], ["GO:0005576", "C:extracellular region", "IEA:InterPro"], ["GO:0019953", "P:sexual reproduction", "IEA:InterPro"]]]
[{}]
[["AT1G07010", "AT4G17030"], ["AT4G17030", "AT3G17860"]]
["GO:0006952 P:defense response"]
["GO:0006952 P:defense response", "GO:0009867 P:jasmonic acid mediated signaling pathway"]
["GO:0006952 P:defense response", "GO:0009867 P:jasmonic acid mediated signaling pathway", "GO:0031347 P:regulation of defense response"

["GO:0030154 P:cell differentiation", "GO:0009908 P:flower development", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation"]
["GO:0030154 P:cell differentiation", "GO:0009908 P:flower development", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation"]
["GO:0030154 P:cell differentiation", "GO:0009908 P:flower development", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation", "GO:0030154 P:cell differentiation"]
[[["GO:0009507", "C:chloroplast", "IDA:UniProtKB"], ["GO:0030145", "F:manganese ion binding", "IDA:UniProtKB"], ["GO:0016791", "F:phosphatase activity", "IDA:UniProtKB"], ["GO:0004721", "F:phosphoprotein phosphatase activity",

[[["GO:0009507", "C:chloroplast", "IDA:UniProtKB"], ["GO:0030145", "F:manganese ion binding", "IDA:UniProtKB"], ["GO:0016791", "F:phosphatase activity", "IDA:UniProtKB"], ["GO:0004721", "F:phosphoprotein phosphatase activity", "IEA:UniProtKB-KW"], ["GO:0005777", "C:peroxisome", "IEA:EnsemblPlants"], ["GO:0016787", "F:hydrolase activity", "IEA:InterPro"], ["GO:0005777", "C:peroxisome", "IDA:TAIR"], ["GO:0016787", "F:hydrolase activity", "IEA:InterPro"], ["GO:0016021", "C:integral component of membrane", "IEA:UniProtKB-KW"], ["GO:0102121", "F:ceramidase activity", "IEA:UniProtKB-UniRule"], ["GO:0017040", "F:N-acylsphingosine amidohydrolase activity", "IEA:UniProtKB-UniRule"], ["GO:0046514", "P:ceramide catabolic process", "IEA:InterPro"], ["GO:0016787", "F:hydrolase activity", "IEA:InterPro"], ["GO:0016787", "F:hydrolase activity", "IEA:InterPro"]], [["GO:0005634", "C:nucleus", "IBA:GO_Central"], ["GO:0042802", "F:identical protein binding", "IPI:IntAct"], ["GO:0006952", "P:defense respo

[{}, {"ath04075"=>"Plant hormone signal transduction"}, {}, {"ath00250"=>"Alanine, aspartate and glutamate metabolism", "ath00760"=>"Nicotinate and nicotinamide metabolism", "ath01100"=>"Metabolic pathways", "ath01240"=>"Biosynthesis of cofactors"}, {}, {}]
[["AT4G17030", "AT1G07010"], ["AT4G17030", "AT3G17860"], ["AT3G17860", "AT1G67170"], ["AT3G17860", "AT5G14760"], ["AT3G17860", "AT1G18660"], ["AT3G17860", "AT5G14070"], ["AT3G17860", "AT4G14465"]]
["GO:0042742 P:defense response to bacterium"]
["GO:0042742 P:defense response to bacterium", "GO:0045087 P:innate immune response"]
["GO:0042742 P:defense response to bacterium", "GO:0045087 P:innate immune response", "GO:1900425 P:negative regulation of defense response to bacterium"]
["GO:0042742 P:defense response to bacterium", "GO:0045087 P:innate immune response", "GO:1900425 P:negative regulation of defense response to bacterium", "GO:0045824 P:negative regulation of innate immune response"]
["GO:0042742 P:defense response to bacte

[{}, {"ath04075"=>"Plant hormone signal transduction"}, {}, {"ath00250"=>"Alanine, aspartate and glutamate metabolism", "ath00760"=>"Nicotinate and nicotinamide metabolism", "ath01100"=>"Metabolic pathways", "ath01240"=>"Biosynthesis of cofactors"}, {}, {}, {}]
[["AT4G17030", "AT1G07010"], ["AT4G17030", "AT3G17860"], ["AT3G17860", "AT1G67170"], ["AT3G17860", "AT5G14760"], ["AT3G17860", "AT1G18660"], ["AT3G17860", "AT5G14070"], ["AT3G17860", "AT4G14465"], ["AT3G17860", "AT4G17220"]]
["GO:0071555 P:cell wall organization"]
["GO:0071555 P:cell wall organization", "GO:0007010 P:cytoskeleton organization"]
["GO:0071555 P:cell wall organization", "GO:0007010 P:cytoskeleton organization", "GO:0009832 P:plant-type cell wall biogenesis"]
["GO:0071555 P:cell wall organization", "GO:0007010 P:cytoskeleton organization", "GO:0009832 P:plant-type cell wall biogenesis", "GO:0010051 P:xylem and phloem pattern formation"]
["GO:0071555 P:cell wall organization", "GO:0007010 P:cytoskeleton organization"

...

[{}, {"ath04075"=>"Plant hormone signal transduction"}, {}, {"ath00250"=>"Alanine, aspartate and glutamate metabolism", "ath00760"=>"Nicotinate and nicotinamide metabolism", "ath01100"=>"Metabolic pathways", "ath01240"=>"Biosynthesis of cofactors"}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {"ath00230"=>"Purine metabolism", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynthesis of secondary metabolites", "ath01240"=>"Biosynthesis of cofactors", "ath03008"=>"Ribosome biogenesis in eukaryotes"}, {"ath00010"=>"Glycolysis / Gluconeogenesis", "ath00020"=>"Citrate cycle (TCA cycle)", "ath00620"=>"Pyruvate metabolism", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynthesis of secondary metabolites", "ath01200"=>"Carbon metabolism"}, {"ath03020"=>"RNA polymerase"}, {}, {}, {"ath00630"=>"Glyoxylate and dicarboxylate metabolism", "ath00710"=>"Carbon fixation in photosynthetic organisms", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynthesis of secondary metabolites", "ath01200"=>"C

["GO:0009734 P:auxin-activated signaling pathway", "GO:0030154 P:cell differentiation", "GO:0009736 P:cytokinin-activated signaling pathway", "GO:0048527 P:lateral root development", "GO:0010078 P:maintenance of root meristem identity", "GO:0010088 P:phloem development", "GO:2000280 P:regulation of root development", "GO:0009737 P:response to abscisic acid", "GO:0048364 P:root development", "GO:0048756 P:sieve cell differentiation", "GO:0048825 P:cotyledon development", "GO:0009736 P:cytokinin-activated signaling pathway", "GO:0009873 P:ethylene-activated signaling pathway", "GO:0048366 P:leaf development", "GO:0019760 P:glucosinolate metabolic process", "GO:1901001 P:negative regulation of response to salt stress", "GO:0010929 P:positive regulation of auxin mediated signaling pathway"]
["GO:0009734 P:auxin-activated signaling pathway", "GO:0030154 P:cell differentiation", "GO:0009736 P:cytokinin-activated signaling pathway", "GO:0048527 P:lateral root development", "GO:0010078 P:maint

[{}, {"ath04075"=>"Plant hormone signal transduction"}, {}, {"ath00250"=>"Alanine, aspartate and glutamate metabolism", "ath00760"=>"Nicotinate and nicotinamide metabolism", "ath01100"=>"Metabolic pathways", "ath01240"=>"Biosynthesis of cofactors"}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {"ath00230"=>"Purine metabolism", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynthesis of secondary metabolites", "ath01240"=>"Biosynthesis of cofactors", "ath03008"=>"Ribosome biogenesis in eukaryotes"}, {"ath00010"=>"Glycolysis / Gluconeogenesis", "ath00020"=>"Citrate cycle (TCA cycle)", "ath00620"=>"Pyruvate metabolism", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynthesis of secondary metabolites", "ath01200"=>"Carbon metabolism"}, {"ath03020"=>"RNA polymerase"}, {}, {}, {"ath00630"=>"Glyoxylate and dicarboxylate metabolism", "ath00710"=>"Carbon fixation in photosynthetic organisms", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynthesis of secondary metabolites", "ath01200"=>"C

["ath04075:Plant hormone signal transduction"]
[{}, {"ath04075"=>"Plant hormone signal transduction"}, {}, {"ath00250"=>"Alanine, aspartate and glutamate metabolism", "ath00760"=>"Nicotinate and nicotinamide metabolism", "ath01100"=>"Metabolic pathways", "ath01240"=>"Biosynthesis of cofactors"}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {}, {"ath00230"=>"Purine metabolism", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynthesis of secondary metabolites", "ath01240"=>"Biosynthesis of cofactors", "ath03008"=>"Ribosome biogenesis in eukaryotes"}, {"ath00010"=>"Glycolysis / Gluconeogenesis", "ath00020"=>"Citrate cycle (TCA cycle)", "ath00620"=>"Pyruvate metabolism", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynthesis of secondary metabolites", "ath01200"=>"Carbon metabolism"}, {"ath03020"=>"RNA polymerase"}, {}, {}, {"ath00630"=>"Glyoxylate and dicarboxylate metabolism", "ath00710"=>"Carbon fixation in photosynthetic organisms", "ath01100"=>"Metabolic pathways", "ath01110"=>"Biosynt

NoMethodError: undefined method `gene_ID' for New:Class

In [ ]:
InteractionNetwork.inspect